In [1]:
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', None)

In [2]:
cellphones = pd.read_csv(r'..\crawling\cellphones\raw_data.csv', encoding='utf-8-sig')
tgdd = pd.read_csv(r'..\crawling\tgdd\raw_data.csv', encoding='utf-8-sig')

In [3]:
cellphones['operating_system'] = cellphones['operating_system'].replace(['Android 16', 'Android 17'], np.nan)


In [4]:
data = pd.concat([cellphones, tgdd], ignore_index=True)

# Xử lí dữ liệu trùng lắp

In [5]:
# Kiểm tra duplicate trong dữ liệu
data.duplicated().sum()

1292

Một số máy có thông số giống nhau hoàn toàn -> Drop.

In [6]:
# Drop các dòng bị duplicated trong dữ liệu
data.drop_duplicates(inplace=True)

# Xử lí các dữ liệu rỗng

In [7]:
# Tìm các dòng rỗng hoàn toàn (tất cả các cột đều là NaN)
empty_rows = data.isnull().all(axis=1)
print(empty_rows.sum())

1


In [8]:
# Drop các dòng rỗng hoàn toàn trong dữ liệu
data.drop(index=data[pd.Series(empty_rows).values].index, inplace=True)

Thuộc tính price_old là thuộc tính rất quan trọng, được xem là target của bài toán -> Drop các dòng bị thiếu thuộc tính price_old

In [9]:
# Loại bỏ các dòng bị thiếu giá trị trong cột 'price_old'
data = data.dropna(subset=['price_old'])

# Xử lí cột name, brand

In [10]:
print(data['name'].unique())

['iPhone 14 Pro Max 128GB' 'iPhone 13 Pro Max 128GB'
 'OPPO Find N3 Flip 12GB 256GB' 'Samsung Galaxy S24 Plus 12GB 256GB'
 'Samsung Galaxy S24 Ultra 12GB 256GB' 'iPhone 14 Pro Max 256GB'
 'iPhone 14 Pro 128GB' 'iPhone 13 Pro Max 256GB' 'iPhone 15 Plus 128GB'
 'Samsung Galaxy S23 Ultra 8GB 256GB' 'OPPO A78 8GB 256GB'
 'OPPO Find N3 16GB 512GB' 'iPhone 15 Pro 128GB' 'iPhone 15 Pro Max 256GB'
 'Samsung Galaxy S23 8GB 128GB'
 'iPhone 16 Plus 128GB Chính hãng VN/A (3N350)' 'iPhone 11 64GB'
 'iPhone 8 Plus 64GB cũ đẹp' 'iPhone 15 Pro 256GB'
 'Samsung Galaxy S22 Ultra (12GB' 'iPhone XS Max 256GB Cũ đẹp'
 'iPhone 12 Pro Max 128GB' 'Samsung Galaxy Z Fold5 12GB 512GB'
 'Apple iPhone 7 Plus 128GB' 'Samsung Galaxy M55 (12GB 256GB)'
 'iPhone 14 128GB' 'iPhone XS Max 64GB Cũ trầy xước' 'iPhone 11 128GB'
 'iPhone 16 Pro Max 256GB | Chính hãng VN/A' 'iPhone 15 128GB'
 'iPhone 14 Plus 128GB' 'iPhone 13 256GB'
 'Samsung Galaxy A35 5G 8GB 128GB' 'iPhone 12 Pro Max 256GB'
 'iPhone 14 Pro 256GB' 'iPhone 12

Cần loại bỏ các thông tin không cần thiết về cấu hình, tình trạng, chức năng. Chỉ giữ lại hãng và model điện thoại.

In [11]:
# Đổi kiểu dữ liệu thành str và chuyển thành chữ thường
data['name'] = data['name'].astype(str).str.lower()

# Xóa "điện thoại "
data['name'] = data['name'].str.replace(r'điện thoại ', '', regex=True)

# Loại bỏ các giá trị không hợp lệ
data = data.drop(data[data['name'] == 'false'].index)

# Xóa thông tin không cần thiết
pattern_to_remove = r'\s*(?:' \
                    r'\d+\s*gb' \
                    r'|\d+\s*ram' \
                    r'|\d+\s*rom' \
                    r'|\d+gb:\d+gb' \
                    r'|\d+:\d+' \
                    r'|cũ' \
                    r'|\d+\s*tb' \
                    r'|đã' \
                    r'|\b\d{4}\b' \
                    r'|2\s*sim' \
                    r'|\(' \
                    r'|chính' \
                    r'|nfc' \
                    r'|vn/a' \
                    r').*$'

data['name'] = data['name'].str.replace(pattern_to_remove, '', regex=True)

# Bổ sung hãng Apple cho các dòng iPhone
data['name'] = data['name'].str.replace(r'\bapple\b', '', regex=True, flags=re.IGNORECASE)
data['name'] = data['name'].str.replace(r'\biphone\b', 'apple iphone', regex=True, flags=re.IGNORECASE)

# Đồng bộ "+" thành "plus"
data['name'] = data['name'].str.replace(r'\s*\+\s*', ' plus ', regex=True)

# **Xóa khoảng trắng dư thừa **
data['name'] = data['name'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [12]:
data_name_unique = sorted(data['name'].dropna().unique())

for name in data_name_unique:
    print(name)

del data_name_unique

apple iphone 11
apple iphone 11 pro
apple iphone 11 pro max
apple iphone 12
apple iphone 12 mini
apple iphone 12 pro
apple iphone 12 pro max
apple iphone 13
apple iphone 13 mini
apple iphone 13 pro
apple iphone 13 pro max
apple iphone 14
apple iphone 14 plus
apple iphone 14 pro
apple iphone 14 pro max
apple iphone 15
apple iphone 15 plus
apple iphone 15 pro
apple iphone 15 pro max
apple iphone 16
apple iphone 16 plus
apple iphone 16 pro
apple iphone 16 pro max
apple iphone 16e
apple iphone 7 plus
apple iphone 8
apple iphone 8 plus
apple iphone se
apple iphone x
apple iphone xr
apple iphone xs
apple iphone xs max
asus rog phone 5s
asus rog phone 6
benco 4g g3
benco s1 pro
benco v91
honor x5 plus
honor x5b
honor x5b plus
honor x6b
honor x7c
honor x8b
honor x8c
honor x9c 5g
honor x9c smart 5g
infinix hot 40 pro free fire
infinix note 30
infinix note 40 pro
inoi 288s 4g
itel it2600
itel it9211
itel it9310
itel p55 plus
itel rs4
masstel fami 50 4g
masstel fami 60s 4g
masstel izi 10
masstel 

Đã chuẩn hóa tương đối tốt tên điện thoại, tuy nhiên "5g" làm một số tên khác nhau chỉ cùng một mẫu điện thoại. Điều này tùy vào nhà sản xuất, một số dòng có phân biệt mẫu 4g và 5g, một số dòng thì không.
Giải pháp là xóa "5g" tùy chọn theo dòng. Vậy cần xác định chính xác hãng và dòng của các điện thoại này. Tiến hành xử lý thủ công để xóa 5g.

In [13]:
# Chữ đầu tiên trong name là brand
data['brand'] = data['name'].str.split(' ').str[0]

In [14]:
#Lọc danh sách thương hiệu có sản phẩm chứa "5g"
brand_5g = data.loc[data['name'].str.contains(r'\b5g\b', regex=True, na=False), 'brand'].dropna().unique()
print(brand_5g)
#Lọc tất cả sản phẩm unique thuộc các thương hiệu trong brand_5g
name_5g_brands = data.loc[data['brand'].isin(brand_5g), 'name'].dropna().unique()
name_5g_brands_sorted = sorted(name_5g_brands)

for name in name_5g_brands_sorted:
    print(name)

['samsung' 'xiaomi' 'oppo' 'vivo' 'poco' 'realme' 'oneplus' 'nubia'
 'honor' 'tecno']
honor x5 plus
honor x5b
honor x5b plus
honor x6b
honor x7c
honor x8b
honor x8c
honor x9c 5g
honor x9c smart 5g
nubia music
nubia neo 2
nubia neo 3 gt
nubia redmagic 8s pro 5g
nubia v60 design
oneplus 11
oneplus 11 5g
oneplus 8t 5g
oneplus nord ce 3 lite
oppo a16
oppo a17k
oppo a18
oppo a3
oppo a38
oppo a3x
oppo a5 pro
oppo a5 pro 5g
oppo a57
oppo a58
oppo a58 4g
oppo a60
oppo a77s
oppo a78
oppo a79 5g
oppo a98
oppo a98 5g
oppo find n2 flip
oppo find n3
oppo find n3 flip
oppo find n3 flip 5g
oppo find n5 fold
oppo find x5 pro
oppo find x5 pro 5g
oppo find x8
oppo find x8 5g
oppo find x8 pro 5g
oppo reno10 5g
oppo reno10 pro plus 5g
oppo reno11 5g
oppo reno11 f 5g
oppo reno11 pro 5g
oppo reno12 5g
oppo reno12 f
oppo reno12 f 5g
oppo reno12 pro 5g
oppo reno13 5g
oppo reno13 f
oppo reno13 f 4g
oppo reno13 f 5g
oppo reno13 pro 5g
oppo reno7
oppo reno8 5g
oppo reno8 pro 5g
poco m6
poco x5 5g
realme 10
realm

không quan tâm các mẫu chỉ tồn tại một phiên bản trong data, không gây khó hiểu (honor, nubia, oneplus), giữ lại toàn bộ các dòng có phân biệt rõ rệt giữa 4g và 5g (tecno, vivo).

In [15]:
# oppo: A series, Reno series, F series, Note series-> giữ lại; Find series -> bỏ
# Lọc các dòng có brand là "oppo" và name chứa "find"
data.loc[(data['brand'].str == 'oppo') & (data['name'].str.contains(r'\bfind\b', regex=True, na=False)), 'name'] = \
    data.loc[(data['brand'].str == 'oppo') & (data['name'].str.contains(r'\bfind\b', regex=True, na=False)), 'name'] \
    .str.replace(r'\b5g\b', '', regex=True) \
    .str.replace(r'\s+', ' ', regex=True) \
    .str.strip()

In [16]:
# samsung: A series, M series -> giữ lại; S series, Note series, Z flip series, Z fold series -> bỏ
# Điều kiện: Chỉ chọn thương hiệu "samsung" và loại trừ A series, M series
condition = (data['brand'] == 'samsung') & ~data['name'].str.contains(r'\ba\d+|\bm\d+', regex=True, na=False)
# Cập nhật dữ liệu: Xóa "5g" và chuẩn hóa khoảng trắng
data.loc[condition, 'name'] = data.loc[condition, 'name'].str.replace(r'\b5g\b', '', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()

In [17]:
# xiaomi: redmi, redmi note -> giữ lại; xiaomi -> bỏ
# Điều kiện: Chỉ áp dụng cho thương hiệu "xiaomi" và lọc các sản phẩm có số (không phải dòng redmi, redmi note, poco)
condition = (data['brand'] == 'xiaomi') & data['name'].str.match(r'\bxiaomi \d+[a-z]*\b', na=False)

# Xóa từ "5g" trong name của các dòng thỏa mãn điều kiện
data.loc[condition, 'name'] = data.loc[condition, 'name'].str.replace(r'\b5g\b', '', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()


# Xử lí missing values

In [18]:
data.isnull().sum()

name                     0
brand                    0
color                    1
condition               35
price_old                0
price_new               26
image                    0
warranty               322
CPU                    136
RAM                    226
capacity                22
time                   442
battery                  3
screen_size              5
operating_system      2019
display_technology     697
screen_resolution      113
SIM                     44
size                    67
weight                1203
bluetooth              279
refresh_rate          2001
GPU                   1818
dtype: int64

Khi cùng một model điện thoại (trong bài là "name"), các yếu tố "CPU", "time", "battery", "screen_size", "operating_system", "display_technology", "screen_resolution", "SIM", "size", "weight", "bluetooth", "refresh_rate", "GPU" thường không đổi chỉ trừ một số phiên bản giới hạn. Vậy có thể điền các giá trị này nếu bị thiếu dựa vào "name".

In [19]:
for name_value, group in data.groupby('name'):
    total = len(group)
    columns_to_check = ["CPU", "time", "battery", "screen_size", "operating_system", 
                    "display_technology", "screen_resolution", "SIM", "size", "weight", "bluetooth", 
                    "refresh_rate", "GPU"]
    
    for column in columns_to_check:
        missing = group[column].isna().sum()
        ratio = missing / total

        if 0 < ratio < 1:
            # Lấy giá trị đầu tiên không bị thiếu trong nhóm
            value = group[column].dropna().iloc[0]
            # Gán lại cho các dòng bị thiếu trong nhóm đó
            data.loc[(data['name'] == name_value) & (data[column].isna()), column] = value

Riêng với RAM và capacity thì có thể thay đổi tùy vào phiên bản điện thoại, vậy có thể kiểm tra xem "name" đó có bao nhiêu giá trị "RAM" và "capacity" duy nhất. Nếu chỉ có một tức là mẫu này chỉ có một phiên bản, vậy hoàn toàn có thể điền thiếu. Nếu có nhiều hơn một phiên bản, vậy dữ liệu này không thể điền, chỉ có thể drop

In [20]:
for name_value, group in data.groupby('name'):
    # Kiểm tra số lượng phiên bản dựa trên RAM và capacity
    unique_ram = group["RAM"].nunique()
    unique_capacity = group["capacity"].nunique()

    # Điền RAM nếu chỉ có một phiên bản
    if unique_ram == 1:
        ram_value = group["RAM"].dropna().iloc[0]
        data.loc[(data['name'] == name_value) & (data["RAM"].isna()), "RAM"] = ram_value
    
    # Điền capacity nếu chỉ có một phiên bản
    if unique_capacity == 1:
        capacity_value = group["capacity"].dropna().iloc[0]
        data.loc[(data['name'] == name_value) & (data["capacity"].isna()), "capacity"] = capacity_value

In [21]:
data.isnull().sum()

name                     0
brand                    0
color                    1
condition               35
price_old                0
price_new               26
image                    0
warranty               322
CPU                    133
RAM                    143
capacity                21
time                   207
battery                  1
screen_size              2
operating_system      1407
display_technology     400
screen_resolution       80
SIM                      9
size                    21
weight                 870
bluetooth              254
refresh_rate          1809
GPU                   1730
dtype: int64

In [22]:
data.shape

(11872, 23)

drop mọi thuộc tính thiếu price_new, RAM, capacity

In [23]:
data = data.dropna(subset=['price_new'])
data = data.dropna(subset=["RAM", "capacity"]) 

đối với condition, warranty không thu thập được, mặc định là "Cũ" và "0 tháng" (không có bảo hành).

In [24]:
data['condition'] = data['condition'].replace('', pd.NA).fillna('Cũ')
data['warranty'] = data['warranty'].replace('', pd.NA).fillna('0 tháng')

In [25]:
data.isnull().sum()

name                     0
brand                    0
color                    1
condition                0
price_old                0
price_new                0
image                    0
warranty                 0
CPU                     17
RAM                      0
capacity                 0
time                   181
battery                  1
screen_size              1
operating_system      1278
display_technology     273
screen_resolution       60
SIM                      9
size                    21
weight                 844
bluetooth              234
refresh_rate          1666
GPU                   1614
dtype: int64

Tất cả missing ratio hiện tại đều là 100% theo name

In [26]:
missing_operating_system_counts = data[data['operating_system'].isna()].groupby('name').size()
total_counts = data.groupby('name').size()

# Tính tỉ lệ thiếu
missing_ratio = (missing_operating_system_counts / total_counts) * 100

# In các dòng bị thiếu
result = pd.DataFrame({'Missing Count': missing_operating_system_counts, 'Missing Ratio (%)': missing_ratio}).dropna()
print(result)

                             Missing Count  Missing Ratio (%)
name                                                         
honor x5 plus                          3.0              100.0
honor x8b                              1.0              100.0
itel it2600                            8.0              100.0
itel it9310                            4.0              100.0
masstel fami 50 4g                   116.0              100.0
masstel fami 60s 4g                   31.0              100.0
masstel izi 10                       137.0              100.0
masstel izi 30 4g                      1.0              100.0
masstel izi t2 4g                      1.0              100.0
masstel izi t6 4g                     81.0              100.0
masstel lux 10                         1.0              100.0
mobell f209                           89.0              100.0
mobell f309 4g                       146.0              100.0
mobell m239 4g                       332.0              100.0
mobell m

Đối với operating_system thường sẽ phụ thuộc vào brand và series. Do trong name đã có sẵn brand, vậy dò các name có độ trùng khớp cao (khả năng cao là cùng series) để điền thiếu. 

In [27]:
from fuzzywuzzy import process

# Lấy danh sách 'name' duy nhất có hệ điều hành
unique_names_os = data.dropna(subset=['operating_system'])[['name', 'operating_system']].drop_duplicates().set_index('name').to_dict()['operating_system']

# Lọc danh sách các tên duy nhất cần điền
missing_names = data.loc[data['operating_system'].isna(), 'name'].unique()

# Tạo ánh xạ giữa tên bị thiếu và tên có hệ điều hành gần nhất (chỉ giữ tên có độ trùng khớp > 80)
name_match_mapping = {
    name: match for name, (match, score) in 
    ((name, process.extractOne(name, unique_names_os.keys())) for name in missing_names) if score > 80
}

# Gán hệ điều hành dựa trên tên gần nhất
data.loc[data['operating_system'].isna(), 'operating_system'] = data['name'].map(lambda x: unique_names_os.get(name_match_mapping.get(x, x), None))

# Tạo bảng thống kê tỷ lệ thiếu
missing_operating_system_counts = data[data['operating_system'].isna()].groupby('name').size()
total_counts = data.groupby('name').size()
missing_ratio = (missing_operating_system_counts / total_counts) * 100

result = pd.DataFrame({'Missing Count': missing_operating_system_counts, 'Missing Ratio (%)': missing_ratio}).dropna()
print(result)

c:\Users\Admin\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


                 Missing Count  Missing Ratio (%)
name                                             
itel it2600                8.0              100.0
itel it9310                4.0              100.0
mobell f209               89.0              100.0
mobell m539               79.0              100.0
mobell rock 4              4.0              100.0
sony xperia 10v            1.0              100.0
tcl 406s                   4.0              100.0


Các dòng điện thoại bị thiếu ở trên đều theo hệ điều hành Android

In [28]:
# gán các giá trị còn lại theo brand
data['operating_system'] = data['operating_system'].replace('', pd.NA).fillna('Android')

Đối với GPU, có thể do điện thoại không có hoặc nhà sản xuất không công bố

In [29]:
# đặt mặc định là 'Unknown'
data['GPU'] = data['GPU'].fillna('Unknown')

In [30]:
# Xóa các dữ liệu bị thiếu không đáng kể
data = data.dropna(subset=['CPU'])
data = data.dropna(subset=['price_new'])
data = data.dropna(subset=['size'])
data = data.dropna(subset=['screen_size'])

Chuẩn hóa screen_size

In [31]:
print(data['screen_size'].unique())

['6.7 inches' '6.8 inches' '6.1 inches' '6.4 inches' '7.82 inches'
 '5.5 inches' '7.6 inches' '6.9 inches' '6.6 inches' '6.56 inches'
 '6.3 inches' '6.74 inches' '6.67 inches' '5.4 inches' '5.8 inches'
 '6.62 inches' '4.7 inches' '6.36 inches' '6.59 inches' '6.5 inches'
 '6.72 inches' '6.73 inches' '6.78 inches' '6.66 inches' '6.2 inches'
 '6.55 inches' '6.64 inches' '6.79 inches' '6.28 inches' '6.88 inches'
 '6.52 inches' '6.44 inches' '6.53 inches' '6.51 inches' '2.4 inches'
 '6.745 inches' '2.0 inches' '8.12 inches' 'Chính 6.7" & Phụ 3.4"' '6.56"'
 '1.77"' '6.2"' '6.4"' '6.7"' '6.8"' '6.6"' '6.5"' 'Chính 7.6" & Phụ 6.3"'
 '6.73"' '6.1"' '1.8"' '6.36"' '2"' '6.67"' '6.79"' '6.71"' '6.88 "'
 '6.43"' '6.72"' '2.4"' '6.74"' '2.31"' '6.68 "' '6.9"' '6.64"' '2.8"'
 '6.745 "' 'Chính 6.8" & Phụ 3.26"' '5.4"' '6.78"' '6.3"' '6.59"' '6.83"'
 '6.77 "']


Đối với refresh_rate mặc định là 60Hz đối với điện thoại cảm ứng (màn hình lớn hơn 3 inch), 30Hz đối với điện thoại không cảm ứng (màn hình nhỏ hơn 3 inch)

In [32]:
data['screen_size'] = data['screen_size'].apply(
    lambda x: x.split('&')[0].split('Chính')[1].strip() 
    if isinstance(x, str) and 'Chính' in x and len(x.split('Chính')) > 1
    else x if isinstance(x, str) else None
)

data['screen_size'] = data['screen_size'].apply(lambda x: (
    float(re.search(r'(\d+\.?\d*)', str(x)).group(1))
    if isinstance(x, str) and re.search(r'(\d+\.?\d*)', str(x))
    else None
))

In [33]:
print(data['screen_size'].unique())

[6.7   6.8   6.1   6.4   7.82  5.5   7.6   6.9   6.6   6.56  6.3   6.74
 6.67  5.4   5.8   6.62  4.7   6.36  6.59  6.5   6.72  6.73  6.78  6.66
 6.2   6.55  6.64  6.79  6.28  6.88  6.52  6.44  6.53  6.51  2.4   6.745
 2.    8.12  1.77  1.8   6.71  6.43  2.31  6.68  2.8   6.83  6.77 ]


In [34]:
data.loc[data['refresh_rate'].isna(), 'refresh_rate'] = data.apply(
    lambda row: 60 if row['screen_size'] > 3 else 30, axis=1
)

In [35]:
data.isnull().sum()

name                    0
brand                   0
color                   1
condition               0
price_old               0
price_new               0
image                   0
warranty                0
CPU                     0
RAM                     0
capacity                0
time                  162
battery                 0
screen_size             0
operating_system        0
display_technology    253
screen_resolution      46
SIM                     2
size                    0
weight                820
bluetooth             210
refresh_rate            0
GPU                     0
dtype: int64

các giá trị còn thiếu còn lại không đáng kể -> drop

In [36]:
data.dropna(inplace=True)

In [37]:
# Xuất DataFrame ra file CSV
data.to_csv('updated_data.csv', index=False, encoding='utf-8-sig')